In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from time import sleep
from pymongo import MongoClient
import re

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [2]:
import datetime as dt
from bs4 import BeautifulSoup as bs
import requests

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
options.add_argument("lang=ko_KR")

In [3]:
client = MongoClient()
db = client.test
results = db.results_twitter

In [4]:
# 크롬 드라이버의 경로
# headless
add = '/home/jin/다운로드/chromedriver'
driver = webdriver.Chrome(add)
# , chrome_options=options)

In [5]:
keywords = '재난' # 키워드
words = ['부산', '서울', '광주', '대구', '세종', '제주', '울산', '인천'] # 지역명
startdate = dt.date(year=2019, month=4, day=1)
untildate = dt.date(year=2019, month=4, day=2)
enddate = dt.date(year=2019, month=4, day=4)

In [6]:
url = 'https://twitter.com/search?l=&q='+ keywords +'%20since%3A'+ str(startdate) +'%20until%3A'+ str(enddate) +'&src=typd'

In [7]:
driver.get(url)

In [9]:
startdate = dt.date(year=2019, month=3, day=1)
untildate = dt.date(year=2019, month=3, day=2)
enddate = dt.date(year=2019, month=4, day=30)

In [10]:
while not enddate == startdate:
    url='https://twitter.com/search?l=&q=' \
        + keywords +'%20since%3A'+ str(startdate) \
        + '%20until%3A'+ str(enddate) +'&src=typd'
    
    driver.get(url)

    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    # 무한 스크롤
    lastHeight = driver.execute_script('return document.body.scrollHeight')
    
    while True:        
        # 무한 스크롤
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(1)
        newHeight = driver.execute_script('return document.body.scrollHeight')
        
        if newHeight == lastHeight:
            html = driver.page_source
            soup = bs(html, 'html.parser')
            tweets = soup.find_all('p', {'class': 'TweetTextSize'})

            for t in tweets:
                result = {
                    'content' : t.text,
                    'date' : str(startdate),
                    'keywords' : keywords
                }
                
                results.insert(result)
                
            startdate = untildate
            untildate += dt.timedelta(days=1)
            
            break
        
        lastHeight = newHeight
    
    

/home/jin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [34]:
for t in results.find().limit(3):
    print(t)

{'_id': ObjectId('5cc726d485861019a68542a2'), 'content': '일부 뇌구조가 비정상적이 토왜들~\n\n“진성호, ‘산불재난사태에, 문재인 대통령 언론사 사장들과 술마셨다’”\n"이언주, \'언론에서(대통령) 지병설, 숙취 의혹을 얘기한다\'”\n\n진성호, 유튜브에서 가짜뉴스 제작해서 먹고사는 머리가 희안한 토착왜구 아니냐? 쓰레기 청소는 절대 선처없이 빠를수록 좋다. pic.twitter.com/7riyGsSpsM', 'date': '2019-04-01', 'keywords': '재난'}
{'_id': ObjectId('5cc726d485861019a68542a3'), 'content': "정문호 소방청장님에게 강원 산불 소방대응단계 발령이 '적절한 조치였냐'고 따져 묻고 있는 이언주. 어따 대고 감히. 야! 이번 강원 산불 대응은 '재난 대응 교본에 대표적 성공 사례로 꼽힐 가능성이 높다'더라. 알고나 좀 떠들어! 아~ 짜증 나. 존재 자체가 악이야 아주! pic.twitter.com/ZdFVXKIXq8", 'date': '2019-04-01', 'keywords': '재난'}
{'_id': ObjectId('5cc726d485861019a68542a4'), 'content': '"부실 재난방송\n시스템 재검토하라"\n문재인 대통령, KBS 질타 pic.twitter.com/4FSzjimvr5', 'date': '2019-04-01', 'keywords': '재난'}


In [11]:
results.count()

/home/jin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


8492